<a href="https://colab.research.google.com/github/srivedya/Finance-Projects-main/blob/main/CAPM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install yfinance statsmodels plotly

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import plotly.express as px

In [ ]:
START = "2018-01-01"
END = None

In [ ]:
# Market proxy (CAPM market portfolio)
MARKET_TICKER = "SPY"        # e.g., SPY, VTI, ^GSPC

# Risk-free proxy (annualized yield in %). Recommended: ^IRX (13-week T-bill)
RF_TICKER = "^IRX"           # e.g., ^IRX. If you don't want RF, set RF_TICKER = None

# Assets you want CAPM for (friendly_name: yahoo_ticker)
ASSET_TICKERS = {
    "ETH": "ETH-USD",
    "NVDA": "NVDA",
    "SILVER": "SI=F",
    "USDINR": "INR=X",
    "USDSGD": "SGD=X",
    # Add/remove freely:
    "BTC": "BTC-USD",
    "GOLD": "GC=F",
    "OIL_WTI": "CL=F",
    "TLT": "TLT",
}

ROLLING_BETA_WINDOW = 90   # trading days (set None to skip rolling beta)
RETURN_FREQ = "D"          # "D" daily or "W" weekly (weekly often gives cleaner betas)


In [ ]:
def download_adj_close(all_tickers, start, end):
    raw = yf.download(all_tickers, start=start, end=end, progress=False, auto_adjust=False)
    if isinstance(raw.columns, pd.MultiIndex):
        px_ = raw["Adj Close"].copy()
    else:
        px_ = raw[["Adj Close"]].rename(columns={"Adj Close": all_tickers[0]})
    return px_.ffill()

def to_returns(prices: pd.DataFrame, freq="D"):
    if freq.upper().startswith("W"):
        prices = prices.resample("W-FRI").last()
    return np.log(prices).diff().dropna()

def rf_to_periodic(rf_level: pd.Series, freq="D"):
    # ^IRX is annualized yield in %. Convert to periodic decimal rate.
    rf_level = rf_level.ffill()
    return (rf_level / 100.0) / (52.0 if freq.upper().startswith("W") else 252.0)

def capm_fit(asset_ret: pd.Series, mkt_ret: pd.Series, rf: pd.Series | None):
    df = pd.DataFrame({"Ri": asset_ret, "Rm": mkt_ret}).dropna()
    if rf is None:
        df["Ri_ex"] = df["Ri"]
        df["Rm_ex"] = df["Rm"]
    else:
        rfx = rf.reindex(df.index).ffill()
        df["Ri_ex"] = df["Ri"] - rfx
        df["Rm_ex"] = df["Rm"] - rfx

    X = sm.add_constant(df["Rm_ex"])
    model = sm.OLS(df["Ri_ex"], X).fit()

    alpha = float(model.params["const"])
    beta  = float(model.params["Rm_ex"])
    r2    = float(model.rsquared)
    p_beta= float(model.pvalues["Rm_ex"])
    nobs  = int(model.nobs)
    return model, alpha, beta, r2, p_beta, nobs, df

def rolling_beta(asset_ret: pd.Series, mkt_ret: pd.Series, rf: pd.Series | None, window: int):
    df = pd.DataFrame({"Ri": asset_ret, "Rm": mkt_ret}).dropna()
    if rf is None:
        df["Ri_ex"] = df["Ri"]
        df["Rm_ex"] = df["Rm"]
    else:
        rfx = rf.reindex(df.index).ffill()
        df["Ri_ex"] = df["Ri"] - rfx
        df["Rm_ex"] = df["Rm"] - rfx

    out, idx = [], []
    for i in range(window, len(df) + 1):
        sub = df.iloc[i-window:i]
        X = sm.add_constant(sub["Rm_ex"])
        fit = sm.OLS(sub["Ri_ex"], X).fit()
        out.append(float(fit.params["Rm_ex"]))
        idx.append(sub.index[-1])
    return pd.Series(out, index=idx)

def interpret_row(asset, alpha_p, beta, r2, p_beta, freq):
    periods = 52 if freq.upper().startswith("W") else 252
    alpha_ann = (1 + alpha_p)**periods - 1

    # Beta interpretation
    if beta >= 1.2:
        beta_msg = f"High market sensitivity (β={beta:.2f}) → tends to amplify market moves."
    elif beta >= 0.8:
        beta_msg = f"Market-like sensitivity (β={beta:.2f}) → moves roughly with the market."
    elif beta >= 0.2:
        beta_msg = f"Low market sensitivity (β={beta:.2f}) → more defensive / less tied to market."
    elif beta >= -0.2:
        beta_msg = f"Near-zero market sensitivity (β={beta:.2f}) → market explains little of this asset."
    else:
        beta_msg = f"Negative beta (β={beta:.2f}) → tends to move opposite the market (hedge-like)."

    # R^2 interpretation
    if r2 >= 0.6:
        r2_msg = f"R²={r2:.2f}: market explains a LOT of this asset’s excess-return variation."
    elif r2 >= 0.3:
        r2_msg = f"R²={r2:.2f}: market explains a meaningful chunk, but other factors matter."
    else:
        r2_msg = f"R²={r2:.2f}: market explains little; asset is driven by other factors/idiosyncratic moves."

    # Alpha interpretation
    if alpha_ann > 0.05:
        alpha_msg = f"Alpha ≈ {alpha_ann*100:.1f}%/yr: outperformed CAPM expectation (if persistent)."
    elif alpha_ann < -0.05:
        alpha_msg = f"Alpha ≈ {alpha_ann*100:.1f}%/yr: underperformed CAPM expectation."
    else:
        alpha_msg = f"Alpha ≈ {alpha_ann*100:.1f}%/yr: close to CAPM expectation."

    # Significance of beta
    if p_beta < 0.01:
        p_msg = f"p(Beta)={p_beta:.3g}: beta is highly statistically significant."
    elif p_beta < 0.05:
        p_msg = f"p(Beta)={p_beta:.3g}: beta is statistically significant."
    else:
        p_msg = f"p(Beta)={p_beta:.3g}: beta is not statistically reliable (noisy estimate)."

    return f"""[{asset}]
- {beta_msg}
- {r2_msg}
- {alpha_msg}
- {p_msg}
"""

In [ ]:
# -------------------------
# 3) DOWNLOAD + RETURNS
# -------------------------
all_tickers = list(ASSET_TICKERS.values()) + [MARKET_TICKER]
if RF_TICKER is not None:
    all_tickers.append(RF_TICKER)

prices = download_adj_close(all_tickers, START, END)

asset_prices = pd.DataFrame(
    {name: prices[tkr] for name, tkr in ASSET_TICKERS.items() if tkr in prices.columns}
).dropna(how="all")

mkt_prices = prices[MARKET_TICKER].dropna()
asset_rets = to_returns(asset_prices, RETURN_FREQ)
mkt_rets   = to_returns(mkt_prices.to_frame("MKT"), RETURN_FREQ)["MKT"]

rf_periodic = None
if RF_TICKER is not None and RF_TICKER in prices.columns:
    rf_level = prices[RF_TICKER]
    if RETURN_FREQ.upper().startswith("W"):
        rf_level = rf_level.resample("W-FRI").last()
    rf_periodic = rf_to_periodic(rf_level, RETURN_FREQ).reindex(asset_rets.index).ffill()

# Align returns
asset_rets = asset_rets.reindex(mkt_rets.index).dropna(how="any")
mkt_rets = mkt_rets.reindex(asset_rets.index)

/usr/local/lib/python3.12/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning:

invalid value encountered in log



In [ ]:
corr = asset_rets.corr()
px.imshow(corr, text_auto=".2f", zmin=-1, zmax=1, aspect="auto",
          title=f"Correlation Heatmap ({'Weekly' if RETURN_FREQ.startswith('W') else 'Daily'} log returns)").update_layout(height=650).show()


In [ ]:
rows = []
models = {}

for asset in asset_rets.columns:
    model, alpha, beta, r2, p_beta, nobs, used_df = capm_fit(asset_rets[asset], mkt_rets, rf_periodic)
    models[asset] = model

    periods = 52 if RETURN_FREQ.upper().startswith("W") else 252
    alpha_ann = (1 + alpha)**periods - 1

    rows.append({
        "Asset": asset,
        "Alpha_per_period": alpha,
        "Alpha_annual_approx": alpha_ann,
        "Beta": beta,
        "R2": r2,
        "p(Beta)": p_beta,
        "Obs": nobs,
    })

capm_table = pd.DataFrame(rows).sort_values("Beta", ascending=False).reset_index(drop=True)
capm_table

,Asset,Alpha_per_period,Alpha_annual_approx,Beta,R2,p(Beta),Obs
0,NVDA,0.000667,0.183037,1.831007,0.484395,0.000000e+00,2913
1,ETH,-0.000048,-0.011908,1.325089,0.090109,9.992987e-62,2913
2,BTC,0.000249,0.064705,0.956266,0.080892,2.478388e-55,2913
3,OIL_WTI,-0.000073,-0.018199,0.567331,0.055954,2.489525e-38,2913
4,SILVER,0.000341,0.089777,0.299333,0.035620,9.195777e-25,2913
5,GOLD,0.000311,0.081500,0.059745,0.005487,6.288894e-05,2913
6,USDINR,0.000013,0.003201,0.017699,0.003104,2.629085e-03,2913
7,USDSGD,-0.000115,-0.028462,0.001279,0.000030,7.674253e-01,2913
8,TLT,-0.000121,-0.030094,-0.120362,0.022182,6.473810e-16,2913


In [ ]:
print("CAPM Interpretation Guide (based on your regression results)\n")
print("Definitions:")
print("- Beta (β): sensitivity to the market. β=1 means market-like, >1 amplifies, <1 defensive, <0 hedge-like.")
print("- Alpha (α): average excess return not explained by market movements (per period; also shown annualized approx).")
print("- R²: fraction of asset excess-return variation explained by the market.")
print("- p(Beta): statistical confidence in β (smaller = more reliable).\n")

for _, r in capm_table.iterrows():
    print(interpret_row(
        r["Asset"],
        r["Alpha_per_period"],
        r["Beta"],
        r["R2"],
        r["p(Beta)"],
        RETURN_FREQ
    ))

CAPM Interpretation Guide (based on your regression results)

Definitions:
- Beta (β): sensitivity to the market. β=1 means market-like, >1 amplifies, <1 defensive, <0 hedge-like.
- Alpha (α): average excess return not explained by market movements (per period; also shown annualized approx).
- R²: fraction of asset excess-return variation explained by the market.
- p(Beta): statistical confidence in β (smaller = more reliable).

[NVDA]
- High market sensitivity (β=1.83) → tends to amplify market moves.
- R²=0.48: market explains a meaningful chunk, but other factors matter.
- Alpha ≈ 18.3%/yr: outperformed CAPM expectation (if persistent).
- p(Beta)=0: beta is highly statistically significant.

[ETH]
- High market sensitivity (β=1.33) → tends to amplify market moves.
- R²=0.09: market explains little; asset is driven by other factors/idiosyncratic moves.
- Alpha ≈ -1.2%/yr: close to CAPM expectation.
- p(Beta)=9.99e-62: beta is highly statistically significant.

[BTC]
- Market-like sen

In [ ]:
if ROLLING_BETA_WINDOW is not None:
    beta_df = pd.DataFrame({
        a: rolling_beta(asset_rets[a], mkt_rets, rf_periodic, ROLLING_BETA_WINDOW)
        for a in asset_rets.columns
    }).dropna(how="all")

    px.line(beta_df, title=f"Rolling CAPM Betas vs {MARKET_TICKER} (window={ROLLING_BETA_WINDOW} periods)")\
      .update_layout(height=600).show()